In [ ]:
import pandas as pd
from keras.src.backend.jax.math import top_k

books=pd.read_csv('books_with_categories.csv')

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None,device=0)
classifier("I love this!")

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764597095549107},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119955107569695},
  {'label': 'fear', 'score': 0.00041385195800103247}]]

In [5]:
predictions=classifier(books["description"][0].split("."))

In [7]:
predictions

[[{'label': 'surprise', 'score': 0.7296026945114136},
  {'label': 'neutral', 'score': 0.1403856873512268},
  {'label': 'fear', 'score': 0.06816215068101883},
  {'label': 'joy', 'score': 0.04794244468212128},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.0026284726336598396},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.4493705630302429},
  {'label': 'disgust', 'score': 0.2735917568206787},
  {'label': 'joy', 'score': 0.10908281058073044},
  {'label': 'sadness', 'score': 0.09362733364105225},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.02697017230093479},
  {'label': 'fear', 'score': 0.006879054941236973}],
 [{'label': 'neutral', 'score': 0.6462163329124451},
  {'label': 'sadness', 'score': 0.2427329272031784},
  {'label': 'disgust', 'score': 0.043422631919384},
  {'label': 'surprise', 'score': 0.028300540521740913},
  {'label': 'joy', 'score': 0.0142114935442

In [9]:
sorted(predictions[0],key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284726336598396},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.04794244468212128},
 {'label': 'neutral', 'score': 0.1403856873512268},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296026945114136}]

In [16]:
import numpy as np

emotion_labels=["anger","disgust","fear","joy","neutral","sadness","surprise"]
isbn=[]
emotion_scores={label:[] for label in emotion_labels}

def calculate_max_emotion_score(predictions):
    per_emotion_scores={label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_prediction[index]["score"])
    return {label:np.max(scores) for label,scores in per_emotion_scores.items()}

In [17]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [18]:
emotion_scores

{'anger': [0.0641336590051651,
  0.612619161605835,
  0.0641336590051651,
  0.3514837622642517,
  0.08141239732503891,
  0.23222453892230988,
  0.5381833910942078,
  0.0641336590051651,
  0.30067020654678345,
  0.0641336590051651],
 'disgust': [0.2735917568206787,
  0.348285049200058,
  0.10400667786598206,
  0.1507224440574646,
  0.18449509143829346,
  0.7271750569343567,
  0.15585480630397797,
  0.10400667786598206,
  0.2794812023639679,
  0.17792759835720062],
 'fear': [0.9281679391860962,
  0.9425276517868042,
  0.9723208546638489,
  0.3607066571712494,
  0.09504341334104538,
  0.051362842321395874,
  0.7474285960197449,
  0.4044962227344513,
  0.9155241250991821,
  0.051362842321395874],
 'joy': [0.9327971935272217,
  0.704420804977417,
  0.7672370672225952,
  0.2518811821937561,
  0.04056444764137268,
  0.0433758907020092,
  0.8725650906562805,
  0.04056444764137268,
  0.04056444764137268,
  0.04056444764137268],
 'neutral': [0.6462163329124451,
  0.8879395127296448,
  0.54947674

In [19]:
from tqdm import tqdm

emotion_labels=["anger","disgust","fear","joy","neutral","sadness","surprise"]
isbn=[]
emotion_scores={label:[] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [03:09<00:00, 27.47it/s]


In [21]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [23]:
emotions_df

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646216,0.967158,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150722,0.360707,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078766,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255170,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459271,0.759456,0.951104,0.368112,0.078766,9788185300535


In [24]:
books=pd.merge(books,emotions_df,on="isbn13")

In [25]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932797,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150722,0.360707,0.251881,0.732685,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459271,0.759456,0.951104,0.368112,0.078766


In [26]:
books.to_csv("books_with_emotions.csv",index=False)